## Recursive Least Square with Kafka and Spark streaming 

This notebook provides an example for estimating the coefficients of a linear model on streaming data coming from a Kafka producer. The coefficient estimation is achieved using the recursive least square (RLS) algorithm, using two different RLS models in parallel (with different forgetting factors).

The linear model has 10 parameters, with coefficients [1,0,0,0,0,0,0,0,0,1] (see notebook KafkaSendRLS).

This notebook uses 
* the [Python client for the Apache Kafka distributed stream processing system](http://kafka-python.readthedocs.io/en/master/index.html) to receive messages from a Kafka cluster. 
* [Spark streaming](https://spark.apache.org/docs/latest/streaming-programming-guide.html) for processing the streaming data



### General import

In [1]:
import time
import re, ast
import numpy as np
import os
import pandas as pd
import findspark
findspark.init()

### Start Spark session


In [2]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[*] pyspark-shell'

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("KafkaReceive") \
    .getOrCreate()

### Connect to Kafka cluster on topic dataLinearModel

In [4]:
#This function creates a connection to a Kafka stream
#You may change the topic, or batch interval
#The Zookeeper server is assumed to be running at 127.0.0.1:2181
#The function returns the Spark context, Spark streaming context, and DStream object
def getKafkaDStream(spark,topic='persistence',batch_interval=10):

    #Get Spark context
    sc=spark.sparkContext

    #Create streaming context, with required batch interval
    ssc = StreamingContext(sc, batch_interval)

    #Checkpointing needed for stateful transforms
    ssc.checkpoint("checkpoint")
    
    #Create a DStream that represents streaming data from Kafka, for the required topic 
    dstream = KafkaUtils.createStream(ssc, "zoo1:2181,zoo2:2181,zoo3:2181", "spark-streaming-consumer", {topic: 1})
    
    return [sc,ssc,dstream]

In [5]:
#Save state in global Python variable
def saveState(rdd):
    global state_global
    if rdd is not None:
        data=rdd.collect()
        state_global=data

In [6]:
def updateFunction(new_values, state): 
    ## RLS update function
    ## Only update with first value of RDD. You should transofrm new_values to array, and update models for all values 
    if (len(new_values)>0):
        
        key=new_values[0][0]
        yx=new_values[0][1]
        i=yx[0]
        y=yx[1]
        x=yx[2:]
        n=len(x)
        
        beta=state[1]
        beta.shape=(n,1)
        V=state[2]
        mu=state[3]
        N=state[5]   ## number of treated samples
        sse=state[4]*N  ## sum of squared errors 
        x.shape=(1,n)
        err=y-x.dot(beta)
        sse=sse+pow(err,2.0)
        V=1.0/mu*(V-V.dot(x.T).dot(x).dot(V)/(1.0+float(x.dot(V).dot(x.T))))
        gamma=V.dot(x.T)
        beta=beta+gamma*err
        
        return (key,beta,V,mu,sse/(N+1.0),N+1)  ## update formula mod1
        
    else:
        return state

### Define streaming pipeline

* We define a stream with two states, for updating two RLS models in paralell. Each state contains a state of variables to keep the state of the model, as well as to keep track of MSE estimates. A state is a list of 5 elements:
    * The first three are beta, V and mu, and define the state of the model (see RLS formulas in course)
    * The last two are an estimate of the MSE of the model, and the number of treated samples 
* We create a DStream, flat map with the sensor ID as key, update state for the stream, and save MSE

In [7]:
import re, ast

interval = 1
n = 20 # number of features
state_global = None

beta1 = np.zeros(n)  ## initial parameter vector for model 1
V1 = np.diag(np.zeros(n)+10) ## initial covariance matrix for model 1
mu1  =1.0 # forgetting factor for model 1

state1 = (1,beta1,V1,mu1,0,0)
state24 = (24,beta1,V1,mu1,0,0)

beta2 = np.zeros(n)  ## initial parameter vector for model 2
V2 = np.diag(np.zeros(n)+1) ## initial covariance matrix for model 2
mu2 = 0.99 # forgetting factor for model 2



[sc,ssc,dstream]=getKafkaDStream(spark=spark,topic='RLSTrain',batch_interval=interval)

#Evaluate input (a list - see KafkaSendRLS notebook)
dstream = dstream.map(lambda x: (x[0], np.array(ast.literal_eval(x[1]))))
dstream = dstream.transform(lambda rdd: rdd.groupByKey())
#dstream.pprint()

dstream=dstream.flatMap(lambda x: [('mod1',(int(x[0]), 1.0*np.array(x[1]))),
                                  ('mod2',(int(x[0]), 1.0*np.array(x[1])))])
dstream.pprint()

initialStateRDD = sc.parallelize([(1, state1),
                                  (24, state24)])

dstream=dstream.updateStateByKey(updateFunction,initialRDD=initialStateRDD)

#Only display beta and error
#beta should converge to [1,0,0,0,0,0,0,0,0,1] (send KafkaSend notebook)
theta = dstream.map(lambda x: x[1][0:2]+x[1][4:6])
theta.foreachRDD(saveState)
theta.pprint()





### Start streaming application

In [7]:
ssc.start()

-------------------------------------------
Time: 2019-05-29 11:03:45
-------------------------------------------

-------------------------------------------
Time: 2019-05-29 11:03:45
-------------------------------------------
('mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0)
('mod2', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0)

-------------------------------------------
Time: 2019-05-29 11:03:46
-------------------------------------------

-------------------------------------------
Time: 2019-05-29 11:03:46
-------------------------------------------
('mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0)
('mod2', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0)

-------------------------------------------
Time: 2019-05-29 11:03:47
-------------------------------------------

-------------------------------------------
Time: 2019-05-29 11:03:47

### Stop streaming

In [8]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

-------------------------------------------
Time: 2019-05-29 11:03:58
-------------------------------------------

-------------------------------------------
Time: 2019-05-29 11:03:58
-------------------------------------------
('mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0)
('mod2', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0)



In [12]:
# predict the 8th day
theta1 = state_global[0][1] # mod1
theta2 = state_global[1][1] # mod21
np.save('theta1_30s_LSTM', theta1)
np.save('theta2_30s_LSTM', theta2)



-------------------------------------------
Time: 2019-05-25 01:27:23
-------------------------------------------
('mod1', array([[  1.37389221e+00],
       [ -2.93782573e-01],
       [  6.12317876e-03],
       [ -6.47114299e-02],
       [  2.22908001e-02],
       [ -3.18480957e-02],
       [  2.74212896e-02],
       [ -1.59672575e-02],
       [ -1.09852668e-02],
       [ -1.31416399e-02],
       [  3.25334759e-04],
       [  3.91225590e-04],
       [  1.35588574e-05]]), array([[  4.27808054e-10]]), 11487)
('mod2', array([[  8.60954641e-01],
       [  6.14571809e-02],
       [  1.02139587e-01],
       [ -6.33937922e-02],
       [ -1.03221213e-01],
       [ -1.99868789e-02],
       [ -8.61998292e-02],
       [  1.21037396e-01],
       [  2.26962394e-02],
       [  8.06930100e-02],
       [  1.18236977e-02],
       [  1.34640546e-02],
       [  3.95843607e-04]]), array([[  2.56278614e-09]]), 11487)

-------------------------------------------
Time: 2019-05-25 01:27:24
-------------------

-------------------------------------------
Time: 2019-05-25 01:27:32
-------------------------------------------
('mod1', array([[  1.37389221e+00],
       [ -2.93782573e-01],
       [  6.12317876e-03],
       [ -6.47114299e-02],
       [  2.22908001e-02],
       [ -3.18480957e-02],
       [  2.74212896e-02],
       [ -1.59672575e-02],
       [ -1.09852668e-02],
       [ -1.31416399e-02],
       [  3.25334759e-04],
       [  3.91225590e-04],
       [  1.35588574e-05]]), array([[  4.27808054e-10]]), 11487)
('mod2', array([[  8.60954641e-01],
       [  6.14571809e-02],
       [  1.02139587e-01],
       [ -6.33937922e-02],
       [ -1.03221213e-01],
       [ -1.99868789e-02],
       [ -8.61998292e-02],
       [  1.21037396e-01],
       [  2.26962394e-02],
       [  8.06930100e-02],
       [  1.18236977e-02],
       [  1.34640546e-02],
       [  3.95843607e-04]]), array([[  2.56278614e-09]]), 11487)

-------------------------------------------
Time: 2019-05-25 01:27:33
-------------------